In [1]:
import pandas as pd

#reading the data 
data = pd.read_csv('healthcare-dataset-stroke-data.csv')
data
len(data)

5110

In [2]:
#checking the datas for NaN values
data.bmi.isnull().value_counts()
#observing data
data.tail()
data.describe()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     5110 non-null   object 
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 479.2+ KB


In [3]:
#Turning the objects into int
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

data['gender'] = le.fit_transform(data['gender'])
data['Residence_type'] = le.fit_transform(data['Residence_type'])
data['ever_married'] = le.fit_transform(data['ever_married'])
data['smoking_status'] = le.fit_transform(data['smoking_status'])
data['work_type'] = le.fit_transform(data['work_type'])

In [4]:
#setting NaN column 
knn_columns = ['age', 'gender', 'avg_glucose_level', 'hypertension', 'heart_disease', 'work_type', 'Residence_type', 'bmi']
data_bmi = data.loc[: , knn_columns]
data_bmi.head()

,age,gender,avg_glucose_level,hypertension,heart_disease,work_type,Residence_type,bmi
0,67.0,1,228.69,0,1,2,1,36.6
1,61.0,0,202.21,0,0,3,0,NaN
2,80.0,1,105.92,0,1,2,0,32.5
3,49.0,0,171.23,0,0,2,1,34.4
4,79.0,0,174.12,1,0,3,0,24.0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   int64  
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   int64  
 6   work_type          5110 non-null   int64  
 7   Residence_type     5110 non-null   int64  
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     5110 non-null   int64  
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(9)
memory usage: 479.2 KB


In [6]:
#Using KNN classifier to train and replace bmi NaN values
from sklearn.impute import KNNImputer
import numpy as np
imp = KNNImputer(missing_values = np.nan, n_neighbors = 5)
#training the data with KNN
imp.fit(data_bmi)


,"missing_values missing_values: int, float, str, np.nan or None, default=np.nanThe placeholder for the missing values. All occurrences of`missing_values` will be imputed. For pandas' dataframes withnullable integer dtypes with missing values, `missing_values`should be set to np.nan, since `pd.NA` will be converted to np.nan.",nan
,"n_neighbors n_neighbors: int, default=5Number of neighboring samples to use for imputation.",5
,"weights weights: {'uniform', 'distance'} or callable, default='uniform'Weight function used in prediction. Possible values:- 'uniform' : uniform weights. All points in each neighborhood are weighted equally.- 'distance' : weight points by the inverse of their distance. in this case, closer neighbors of a query point will have a greater influence than neighbors which are further away.- callable : a user-defined function which accepts an array of distances, and returns an array of the same shape containing the weights.",'uniform'
,"metric metric: {'nan_euclidean'} or callable, default='nan_euclidean'Distance metric for searching neighbors. Possible values:- 'nan_euclidean'- callable : a user-defined function which conforms to the definition of ``func_metric(x, y, *, missing_values=np.nan)``. `x` and `y` corresponds to a row (i.e. 1-D arrays) of `X` and `Y`, respectively. The callable should returns a scalar distance value.",'nan_euclidean'
,"copy copy: bool, default=TrueIf True, a copy of X will be created. If False, imputation willbe done in-place whenever possible.",True
,"add_indicator add_indicator: bool, default=FalseIf True, a :class:`MissingIndicator` transform will stack onto theoutput of the imputer's transform. This allows a predictive estimatorto account for missingness despite imputation. If a feature has nomissing values at fit/train time, the feature won't appear on themissing indicator even if there are missing values at transform/testtime.",False
,"keep_empty_features keep_empty_features: bool, default=FalseIf True, features that consist exclusively of missing values when`fit` is called are returned in results when `transform` is called.The imputed value is always `0`... versionadded:: 1.2",False


In [ ]:
#Replacing the NaN values with predicted values using transform based off of 5 nearest neighbors and creating a new DataFrame for it
data_bmi = imp.transform(data_bmi)
data_bmi = pd.DataFrame(data_bmi, columns = ['age', 'gender', 'avg_glucose_level', 'hypertension', 'heart_disease', 'work_type', 'Residence_type', 'bmi'])
data_bmi

In [ ]:
data_bmi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   age                5110 non-null   float64
 1   gender             5110 non-null   float64
 2   avg_glucose_level  5110 non-null   float64
 3   hypertension       5110 non-null   float64
 4   heart_disease      5110 non-null   float64
 5   work_type          5110 non-null   float64
 6   Residence_type     5110 non-null   float64
 7   bmi                5110 non-null   float64
dtypes: float64(8)
memory usage: 319.5 KB


In [ ]:
#adding back/ merging the bmi replaced datas back into the original Dataframe to get a clean data
data['bmi'] = data_bmi['bmi']
data

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,1,67.0,0,1,1,2,1,228.69,36.60,1,1
1,51676,0,61.0,0,0,1,3,0,202.21,33.88,2,1
2,31112,1,80.0,0,1,1,2,0,105.92,32.50,2,1
3,60182,0,49.0,0,0,1,2,1,171.23,34.40,3,1
4,1665,0,79.0,1,0,1,3,0,174.12,24.00,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,0,80.0,1,0,1,2,1,83.75,27.10,2,0
5106,44873,0,81.0,0,0,1,3,1,125.20,40.00,2,0
5107,19723,0,35.0,0,0,1,3,0,82.99,30.60,2,0
5108,37544,1,51.0,0,0,1,2,0,166.29,25.60,1,0


In [ ]:
#seperating x and y to train_test and split
factors = ['gender', 'age', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type', 'avg_glucose_level', 'bmi', 'smoking_status'] 
x = data[factors]
data.stroke = data.stroke.astype(int)
y = data.stroke

In [ ]:
from sklearn.model_selection import train_test_split
train_x, val_x, train_y, val_y = train_test_split(x,y, random_state = 1, stratify = y)

In [ ]:
'''
#Got 90.5% accuracy on DecisionTree
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(random_state = 1)
model.fit(train_x, train_y)
prediction =  model.predict(val_x)
prediction = prediction.astype(int)
'''

'\n#Got 90.5% accuracy on DecisionTree\nfrom sklearn.tree import DecisionTreeClassifier\nmodel = DecisionTreeClassifier(random_state = 1)\nmodel.fit(train_x, train_y)\nprediction =  model.predict(val_x)\nprediction = prediction.astype(int)\n'

In [ ]:
#using RandomForest to fit and predict the accuracy
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(random_state = 1)
rf_model.fit(train_x, train_y)
val_prediction = rf_model.predict(val_x)
val_prediction = val_prediction

In [ ]:
from sklearn.metrics import accuracy_score
#tree = accuracy_score(val_y, prediction)
forest = accuracy_score(val_y, val_prediction)
print(f"RandomForest accuracy: {forest}")

RandomForest accuracy: 0.9483568075117371


In [ ]:
len(data)

5110